In [ ]:
# execute script
# for multiple processes, it needs to start outside of notebook
# 225GB ~ 40min
# 240GB ~ 74min ~80k files
from os.path import abspath, join
from subprocess import Popen, PIPE

script_path = abspath('')
script_exec_path = join(script_path,'compare-folders.py')
print("To execute:\n" + script_exec_path)

process = Popen(
    ['py', script_exec_path,
     '-d',
     'E:\\' 
    ],
    stdout=PIPE, stderr=PIPE, 
    text=True  # Ensures strings instead of bytes
)

stdout, stderr = process.communicate()

print("Output:\n", stdout)
print("Error:\n", stderr)
# output: csv file

In [ ]:
from os.path import isdir, abspath
from os import remove, listdir

script_path = abspath('')
keyword = "processed"
in_path = ""
for entry in listdir(script_path):
    if not isdir(entry):
        if ".csv" in entry:
            if keyword in entry:
                remove(entry)
            else:
                in_path = entry
        
print(in_path)
# output: in_path stores location of csv

In [ ]:
# post process csv file
# output: two csv-s
from ast import literal_eval
from csv import DictReader, field_size_limit
from os.path import splitdrive
from sys import maxsize

field_size_limit(maxsize)

sep = '.'
p,ext = in_path.split(sep)
duplicates_path = p + "-" + keyword + sep + ext
singles_path = p + "-" + keyword + "-" + "single" + sep + ext
print(singles_path, duplicates_path)

with open(duplicates_path, mode = "w", encoding="utf-8") as d_file:
    with open(singles_path, mode = "w", encoding="utf-8") as s_file:
        with open(in_path, mode = "r", encoding="utf-8") as in_file:
            reader = DictReader(in_file)
            for i, row in enumerate(reader):
                value = literal_eval(row["paths"])
                if "desktop.ini" in value[0]: # ignore system files
                    continue
            
                # write duplicate paths once:
                s = set()
                for entry in value:
                    drive, entry = splitdrive(entry)
                    s.add(entry)

                if len(s) <= 1:
                    file = s_file
                else: # >=2 are duplicates
                    file = d_file

                file.write(f"{i}\n")
                for entry in s:
                    file.write(f"{join(drive, entry)}\n")


In [ ]:
# deletes duplicates, but one is kept, having the sortest name
from ast import literal_eval
from csv import DictReader, field_size_limit
from os import remove
from os.path import splitdrive, basename
from sys import maxsize
from math import inf

def execute():
    
    with open(in_path, mode = "r", encoding="utf-8") as in_file:
        reader = DictReader(in_file)
        for row in reader:
            value = literal_eval(row["paths"])
            if "desktop.ini" in value[0]: # ignore system files
                continue
        
            # write duplicated paths once:
            set_of_files = set()
            for entry in value:
                drive, entry = splitdrive(entry)
                set_of_files.add(entry)
            set_of_files = sorted(set_of_files)

            # if there are at least two duplicated files, keep only one of them
            if len(set_of_files) > 1: 
                # it keeps the file having the shortest name and then path
                name_to_keep = (inf, 0)
                for i, entry in enumerate(set_of_files):
                    l = len(basename(entry))
                    if l < name_to_keep[0]:
                        name_to_keep = (l, i)
                
                for i, entry in enumerate(set_of_files):
                    file_path = join(drive, entry)
                    print(i + 1)
                    if i != name_to_keep[1]:
                        try:
                            remove(file_path)
                        except Exception as e:
                            print(f"{file_path} error:\n{e}")
                            pass
                    else:
                        print(f" From {len(set_of_files)} duplicates, " + 
                              f"{file_path} file #{i + 1} is kept")

if __name__ == "__main__":

    field_size_limit(maxsize) 

    execute()